# Happy Gates

## Лабораторная работа по [когнитивным сервисам](http://aka.ms/coserv)

В ходе этой работы мы познакомимся с одним из когнитивных сервисов Microsoft - [Face API](https://aka.ms/cognitive_face). Наша цель - найти самые веселые, испуганные, удивлённые фотографии Билла Гейтса (или любой другой знаменитости).

Для этого мы будем использовать [Bing Image Search](https://azure.microsoft.com/en-us/services/cognitive-services/bing-image-search-api/?WT.mc_id=rus-github-dmitryso) для поиска изображений, и Face API для выделения эмоций.

### Получение ключей доступа к когнитивным сервисам

Для использования Face API нам необходим специальный ключ, а также интернет-адрес для вызова сервиса (Endpoint URL). Есть несколько способов получить ключ:

* Если у Вас есть подписка Microsoft Azure, [создайте ресурс Face API](https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-apis-create-account/?wt.mc_id=happygates-github-dmitryso), и скопируйте ключ и URL оттуда.
* Вы всегда можете [создать бесплатную пробную подписку](https://azure.microsoft.com/free/?wt.mc_id=happygates-github-dmitryso), по которой в течение месяца вы сможете расходовать определенный предоставленный вам кредитный лимит, а затем в течение года получать ограниченный набор сервисов бесплатно (для этого понадобится кредитная карта)
* Если у Вас нет облачной подписки, Вы можете попробовать Face API бесплатно в течение 7 дней - запросите ключ [здесь](https://azure.microsoft.com/try/cognitive-services/my-apis/?api=face-api&wt.mc_id=happygates-github-dmitryso). Понадобится только Microsoft Account, который можно бесплатно завести на http://outlook.com. 

**Важно:** При использовании пробного ключа, у Face API будут некоторые ограничения по количеству изображений, которые вы сможете обработать (несколько тысяч), а также по частоте запросов (не более 20 запросов в минуту). Для обеспечения работы без ошибок, возможно придется вставлять паузы между запросами к сервису.

Аналоничным образом нужно будет получить ключ для [Bing Image Search]().

Полученные ключи и адрес Endpoint URL введите в ячейке ниже:

In [11]:
search_key = '-- INSERT YOUR KEY HERE --'
search_url = 'https://api.cognitive.microsoft.com/bing/v5.0/' # обязателен символ '/' в конце
face_api_key = '-- INSERT YOUR KEY HERE --'
face_api_url = 'https://westeurope.api.cognitive.microsoft.com/face/v1.0/' # обязателен символ '/' в конце

import requests, base64

def search(term):
    headers = {
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': search_key }
    params = { 'q': term }

    response = requests.get(url = search_url+"images/search",
                             headers = headers,
                             params = params)
    return response.json()["value"]

Теперь найдём 20 фотографий Билла Гейтса. Аргумент у функции поиска указывается в формате *URL-encoded* для простоты, т.е. спецсимволы лучше не использовать, а пробелы заменяем на `+`.

In [8]:
image_url = [ x["contentUrl"] for x in search("Bill+Gates")]

from IPython.core.display import HTML
HTML("".join([ "<img src=""{}"" width=200 height=200/>".format(image_url[i]) for i in range(3)]))

<Response [200]>


Описываем функцию вызова **Face API** - это обычный вызов интернет-сервиса REST, в который мы передаём ключ в виде параметра заголовка `Ocp-Apim-Subscription-Key`. Процесс создания такого запроса более подробно описан [в документации](https://docs.microsoft.com/azure/cognitive-services/face/quickstarts/python/?WT.mc_id=happygates-github-dmitryso)

In [15]:
def emotion(url):
    headers = {
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': face_api_key }
    
    params = {
    'returnFaceId': 'false',
    'returnFaceLandmarks': 'false',
    'returnFaceAttributes': 'emotion' }
    
    response = requests.post(url = face_api_url+'detect',
                             headers = headers,
                             params = params,
                             json = {'url': url})
    return response.json()

emotion(image_url[0])

[{'faceRectangle': {'top': 449, 'left': 1408, 'width': 631, 'height': 631},
  'faceAttributes': {'emotion': {'anger': 0.0,
    'contempt': 0.0,
    'disgust': 0.0,
    'fear': 0.0,
    'happiness': 1.0,
    'neutral': 0.0,
    'sadness': 0.0,
    'surprise': 0.0}}}]

Самое интересное: вызываем анализ для всех наших фотографий и собираем результаты в словарь самых сильных эмоций. Поскольку у Face API есть ограничение на число вызовов в секунду, то обрабатываем только некоторое количество изображений. Для обработки всех надо добавить задержку между вызовами.

In [20]:
emotions = {'anger':0,'contempt':0,'disgust':0,'fear':0,'happiness':0,'neutral':0,'sadness':0,'surprise':0}
urls = dict.fromkeys(emotions.keys())
for url in image_url[0:9]:
    s = emotion(url)
    print('Processing {}'.format(url))
    if len(s)>0 and not 'error' in s:
        s = s[0]["faceAttributes"]["emotion"]
        for z in emotions.keys():
                if s[z]>emotions[z]:
                    emotions[z] = s[z]
                    urls[z] = url
print(emotions)

Processing http://element9.files.wordpress.com/2008/06/06-15gates_lg.jpg
Processing https://timedotcom.files.wordpress.com/2018/04/bill-gates.jpeg?quality=85
Processing http://i.huffpost.com/gen/1687322/images/o-BILL-GATES-SMILING-facebook.jpg
Processing https://timedotcom.files.wordpress.com/2018/09/bill-gates-africa.jpg
Processing http://www.geofffox.com/wp-content/uploads/2010/09/bill_gates.jpg
Processing http://1.bp.blogspot.com/-o16ik-B_FcE/Tq1-W_OpEfI/AAAAAAAAA4o/6RclKXCjjCc/s1600/Bill+Gates+I.jpg
Processing https://fortunedotcom.files.wordpress.com/2017/04/bill-gates-donald-trump-foreign-aid.jpg
Processing http://www.popsci.com/sites/popsci.com/files/bill_gates_july_2014.jpg
Processing https://cdn.vox-cdn.com/thumbor/YAXrxU8-yIxSt9nm3nvba3SeQHk=/0x0:3000x2000/1200x800/filters:focal(1256x678:1736x1158)/cdn.vox-cdn.com/uploads/chorus_image/image/63137392/1057615164.jpg.0.jpg
{'anger': 0, 'contempt': 0.001, 'disgust': 0, 'fear': 0, 'happiness': 1.0, 'neutral': 0.979, 'sadness': 0.0

In [24]:
HTML("<table>"+ \
      "".join(["<tr><td>{}</td><td><img src=""{}"" width=250 height=250/></td></tr>".format(z,urls[z]) \
               for z in emotions.keys() if emotions[z]>0])+ \
     "</table>")

contempt,
happiness,
neutral,
sadness,
surprise,


## Задание

1. Добавьте задержку и обработайте все фотографии
2. Используйте другую знаменитость (рекомендация - **Стив Баллмер**)
3. *(сложно)* Используйте собранные вами фотографии для определения:
   - среднего уровня счастья для мужчин и женщин
   - распределение уровня счастья по возрасту